In [1]:
import findspark
#useful to install this tool to simplify spark import
findspark.init()
from pyspark import  SparkContext
sc = SparkContext( 'local', 'pyspark')

In [3]:
import numpy
rdd=sc.textFile('train_no_header.csv')

#obtaining rdd: itemid,rating (users are not useful for this recommendation)
def firstMap (line):
    x=line.split(',')
    return x[1],int(x[2])
item_rdd = rdd.map(firstMap).cache()

#computing average vote of the entire dataset
totalVotes = item_rdd.map(lambda x: x[1]).reduce(lambda x,y: x+y)
avgVote = float(totalVotes)/rdd.count()

#creating rdd: itemid, [rating_list]
itemVotes=item_rdd.groupByKey()
itemVotes=itemVotes.map(lambda x:(x[0],list(x[1])))
itemAverage=itemVotes.map(lambda x: (x[0],float(sum(x[1]))/len(x[1]),len(x[1]))).cache()

#computing average number of vote for item of the entire dataset
avgVoteNumberPerItem=float(item_rdd.count())/itemVotes.count()
print avgVoteNumberPerItem
print avgVote

5.07044729863
6.81887639657


In [4]:
#Computing ranking value according to:
#https://www.quora.com/How-is-a-Movies-weighted-rating-calculated-on-IMDb
minimumVotesRequired=avgVoteNumberPerItem*10

def tenStarScore (x):
    R=x[1]
    v=x[2]
    m=minimumVotesRequired
    C=avgVote
    return x[0],(v*R+m*C)/(v+m),x[2]
    
itemRank=itemAverage.map(tenStarScore)            

print minimumVotesRequired
top250=itemRank.takeOrdered(250,key=lambda x: -x[1])
top250

50


[(u'33173', 8.651107541877161, 125),
 (u'33475', 8.548719834659863, 72),
 (u'35300', 8.346038038094793, 103),
 (u'1076', 8.340172607452923, 88),
 (u'31992', 8.32013160126245, 131),
 (u'15743', 8.311550558628026, 75),
 (u'36872', 8.306644250618136, 77),
 (u'12866', 8.286351616386055, 86),
 (u'17648', 8.286351616386055, 86),
 (u'18200', 8.275728956003087, 113),
 (u'34517', 8.263998510979366, 71),
 (u'32347', 8.262665086214941, 64),
 (u'9577', 8.258121223323153, 97),
 (u'24639', 8.244259890372302, 134),
 (u'29848', 8.229591325915958, 63),
 (u'6461', 8.22877652658683, 94),
 (u'11259', 8.224230578467417, 88),
 (u'3431', 8.218805437092135, 64),
 (u'10446', 8.207943193253495, 94),
 (u'25363', 8.201261577443011, 64),
 (u'29915', 8.188683061518047, 61),
 (u'13502', 8.172216543895484, 60),
 (u'14616', 8.14348071819067, 89),
 (u'30360', 8.140562549184587, 99),
 (u'27569', 8.131767106020687, 71),
 (u'2041', 8.12664254091952, 68),
 (u'23188', 8.100368233852969, 69),
 (u'10200', 8.100315250564773, 8

In [5]:
rdd=sc.textFile('train_no_header.csv')
rdd = rdd.map( lambda x: x.split(',') )
rdd = rdd.map(lambda x:(x[0],[x[1],x[2]]))

rdd = rdd.groupByKey()
URM = rdd.map(lambda x:(x[0],list(x[1])))
URM.takeOrdered(10)




[(u'1',
  [[u'3456', u'7'],
   [u'3527', u'10'],
   [u'3864', u'8'],
   [u'3920', u'10'],
   [u'5248', u'10'],
   [u'6111', u'10'],
   [u'6249', u'7'],
   [u'6703', u'8'],
   [u'7393', u'9'],
   [u'11618', u'1'],
   [u'14227', u'5'],
   [u'16571', u'10'],
   [u'19022', u'9'],
   [u'19478', u'4'],
   [u'19629', u'8'],
   [u'20484', u'9'],
   [u'21101', u'10'],
   [u'21195', u'2'],
   [u'22932', u'9'],
   [u'23507', u'10'],
   [u'25533', u'6'],
   [u'27660', u'10'],
   [u'28083', u'10'],
   [u'30478', u'3'],
   [u'33173', u'10'],
   [u'33213', u'9'],
   [u'33603', u'10']]),
 (u'10', [[u'17463', u'5'], [u'21660', u'9']]),
 (u'100', [[u'8013', u'9'], [u'12255', u'2'], [u'30796', u'10']]),
 (u'1000', [[u'3545', u'9'], [u'15593', u'9'], [u'29915', u'10']]),
 (u'10000',
  [[u'2339', u'10'],
   [u'4926', u'10'],
   [u'7992', u'3'],
   [u'9012', u'10'],
   [u'13998', u'10'],
   [u'14532', u'2'],
   [u'17207', u'1'],
   [u'17988', u'1'],
   [u'18669', u'3'],
   [u'19907', u'2'],
   [u'21101', u'

In [6]:
test_users=sc.textFile('test_no_header.csv')

test_list=test_users.collect()

def filter_test_users(x):
    return x[0] in test_list

test_URM=URM.filter(filter_test_users)

print test_URM.count()

test_URM.take(10)

def top_recommendations(x):
    y=[]

    for item in top250:
        if item[0] not in x[1]:
            y.append(item[0])
        else:
            print "ALLELUIA"
        if len(y)>=5: 
            return x[0],y
    return x[0],y
        
recommendations=test_URM.map(top_recommendations)
recommendations.take(10)

def map_string (x):
    y=x[1]
    s=y[0]+" "+y[1]+" "+y[2]+" "+y[3]+" "+y[4]
    return x[0],s
formatted_recommendations=recommendations.map(map_string).takeOrdered(4196,key=lambda x: int(x[0]))
len(formatted_recommendations)


4196


4196

In [8]:
import csv
f1= open('submission5.csv','w+')

csv_f1=csv.writer(f1)
csv_f1.writerow(["userId","testItems"])
for i in formatted_recommendations:
    csv_f1.writerow(i)